In [9]:
import itertools
import random
from copy import deepcopy

random.seed(48)

# # Start with optimal tiling for a 5x5 bingo board
OPTIMAL_BOARD = [['GG', 'GB', 'GB', 'GR', ''],
                 ['GR', 'BR', 'GB', 'BB', ''],
                 ['GG', 'BR', 'BB', 'BR', ''],
                 ['GG', 'BB', 'GR', 'BR', ''], 
                 ['', '', '', '', '']]



In [10]:
# Creating list for all the possible rolls. Occurences in list represent probabilities of being selected
possible_rolls = ["GG"] * 9 + ["GB"] * 12 + ["GR"] * 6 + ["BB"] * 4 + ["BR"] * 4 + ["RR"]
rolls = set(possible_rolls)
# Get all possible combinations for the last 9 cells
b = list(itertools.product(rolls, repeat = 9))


In [11]:
b = list(filter(lambda x : "RR" in x and "GG" in x and "BR" in x and len(set(x)) >= 4 , b ))

In [12]:
index = 1
# Fill in remaining 7 spots on the board
def get_boards(b):
    for vals in b:
        board = deepcopy(OPTIMAL_BOARD)
        board[0][4] = vals[0]
        board[1][4] = vals[1]
        board[2][4] = vals[2]
        board[3][4] = vals[3]
        board[4][0] = vals[4]
        board[4][1] = vals[5]
        board[4][2] = vals[6]
        board[4][3] = vals[7]
        board[4][4] = vals[8]

        yield board

In [13]:
boards = get_boards(b)

In [14]:
# Function to check if board has bingo, X's in the rows, cols, on on diagonals
def check_bingo(board):
    for row in range(len(board)):
        if all(board[row][col] == "X" for col in range(5)) or all(board[col][row] == "X" for col in range(5)):
            return True
    if all(board[i][i] == "X" for i in range(5)) or all(board[i][4-i] == "X" for i in range(5)):
        return True
    return False

# Checks to see if the current tile picked can be placed on bingo board
def check_placement(board, pick):
    for i in range(len(board)):
        for j in range(len(board[0])):
            if board[i][j] == pick:
                board[i][j] = "X"
                return 

# Simulation to take turns for the game
def take_turns(board):
    tries = 0
    while not check_bingo(board):
        tries += 1
        pick = random.choice(possible_rolls)
        check_placement(board, pick)
        if tries > 20:
            return tries
    return tries

In [15]:
def check_better(bb, board, turns):
    worst = [-1000, 0] # Score : Index
    for index, tup in enumerate(bb):
        if tup[0] > worst[0]:
            worst[0] = tup[0]
            worst[1] = index
    if turns < worst[0]:
        bb[worst[1]] = (turns, board)

def check_worse(wb, board, turns):
    best_worst = [1000, 0] # Score : Index
    for index, tup in enumerate(wb):
        if tup[0] < best_worst[0]:
            best_worst[0] = tup[0]
            best_worst[1] = index
    if turns > best_worst[0]:
        wb[best_worst[1]] = (turns, board)

In [17]:
# Do 100,000 simulations for each board and calculate average turns taken
best_boards = [(i+100, None) for i in range(5)]
worst_boards = [(i-100, None) for i in range(5)]
while True:
    try:
        board = next(boards)
        avg_turns = [take_turns(deepcopy(board)) for _ in range(3)]
        avg_turns = sum(avg_turns) / len(avg_turns)
        check_better(best_boards, board, avg_turns)
        check_worse(worst_boards, board, avg_turns)
    except StopIteration:
        break
best_boards

In [2]:
x = [(9.02, [['GG', 'GB', 'GB', 'GR', 'GG'], ['GR', 'BR', 'GB', 'BB', 'GR'], ['GG', 'BR', 'BB', 'BR', 'BR'], ['GG', 'BB', 'GR', 'BR', 'RR'], ['GG', 'BR', 'BB', 'GR', 'GB']]), 
 (8.99, [['GG', 'GB', 'GB', 'GR', 'GG'], ['GR', 'BR', 'GB', 'BB', 'GB'], ['GG', 'BR', 'BB', 'BR', 'BR'], ['GG', 'BB', 'GR', 'BR', 'RR'], ['GR', 'GB', 'GB', 'GB', 'GB']]), 
 (9.02, [['GG', 'GB', 'GB', 'GR', 'GG'], ['GR', 'BR', 'GB', 'BB', 'GB'], ['GG', 'BR', 'BB', 'BR', 'GG'], ['GG', 'BB', 'GR', 'BR', 'RR'], ['BB', 'BR', 'GR', 'GG', 'RR']]), 
 (8.98, [['GG', 'GB', 'GB', 'GR', 'GG'], ['GR', 'BR', 'GB', 'BB', 'BB'], ['GG', 'BR', 'BB', 'BR', 'BR'], ['GG', 'BB', 'GR', 'BR', 'BR'], ['RR', 'GR', 'BB', 'GG', 'GG']]), 
 (8.92, [['GG', 'GB', 'GB', 'GR', 'GB'], ['GR', 'BR', 'GB', 'BB', 'RR'], ['GG', 'BR', 'BB', 'BR', 'BB'], ['GG', 'BB', 'GR', 'BR', 'BB'], ['GG', 'BR', 'BR', 'GB', 'RR']])]
sorted(x)

[(8.92,
  [['GG', 'GB', 'GB', 'GR', 'GB'],
   ['GR', 'BR', 'GB', 'BB', 'RR'],
   ['GG', 'BR', 'BB', 'BR', 'BB'],
   ['GG', 'BB', 'GR', 'BR', 'BB'],
   ['GG', 'BR', 'BR', 'GB', 'RR']]),
 (8.98,
  [['GG', 'GB', 'GB', 'GR', 'GG'],
   ['GR', 'BR', 'GB', 'BB', 'BB'],
   ['GG', 'BR', 'BB', 'BR', 'BR'],
   ['GG', 'BB', 'GR', 'BR', 'BR'],
   ['RR', 'GR', 'BB', 'GG', 'GG']]),
 (8.99,
  [['GG', 'GB', 'GB', 'GR', 'GG'],
   ['GR', 'BR', 'GB', 'BB', 'GB'],
   ['GG', 'BR', 'BB', 'BR', 'BR'],
   ['GG', 'BB', 'GR', 'BR', 'RR'],
   ['GR', 'GB', 'GB', 'GB', 'GB']]),
 (9.02,
  [['GG', 'GB', 'GB', 'GR', 'GG'],
   ['GR', 'BR', 'GB', 'BB', 'GB'],
   ['GG', 'BR', 'BB', 'BR', 'GG'],
   ['GG', 'BB', 'GR', 'BR', 'RR'],
   ['BB', 'BR', 'GR', 'GG', 'RR']]),
 (9.02,
  [['GG', 'GB', 'GB', 'GR', 'GG'],
   ['GR', 'BR', 'GB', 'BB', 'GR'],
   ['GG', 'BR', 'BB', 'BR', 'BR'],
   ['GG', 'BB', 'GR', 'BR', 'RR'],
   ['GG', 'BR', 'BB', 'GR', 'GB']])]